### Timelapse notebook

In [ ]:
from timelapse import *

In [ ]:
sys.path.insert(1, '../source/python')
output_dir = '../output/'
os.makedirs(output_dir,exist_ok=True)

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 150)
pd.set_option('display.width', 100)
plt.rcParams["figure.figsize"] = (12,6)

In [ ]:
file_roma_standardized = '../data/temp_data/ro_standard.xlsx'
file_roma_IDA = '../data/roma_blind_prediction/ID segmentali con IDA Score.xlsx'
file_roma_blind = '../data/roma_blind_prediction/inviato a matteo_v3.xlsx'
file_roma_blind_correction = '../data/roma_blind_prediction/segmentali_per_silvia_SC_MF.xlsx'
file_valencia_standardized = '../data/temp_data/va_standard.xlsx'
file_uk_standardized = '../data/temp_data/uk_standard.xlsx'
file_bologna = '../data/input_data/9.baby TLM Mophokinetics Parameters.xlsx'

In [ ]:
# definire i parametri utilizzati
selected_feature_list = ['tPNa_imp','t3_imp','t5_imp','t6_imp','t7_imp','t8_imp','t9_imp','cc3_imp', 's3_imp','blast_imp']
raw_feature_list = ['tPNa', 'tPNf', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 'tM', 'tSB', 'tB', 'tEB']
# parametri per dataset Genera aggiuntivi
grading_dict = {'AA':'AA','AB':'AB,BA','BA':'AB,BA','AC':'AC,CA,BB','CA':'AC,CA,BB','BB':'AC,CA,BB','CB':'CC,BC,CB','BC':'CC,BC,CB','CC':'CC,BC,CB','-':'-'}
grading_dict_num = {'AA':0,'AB,BA':1,'AC,CA,BB':2,'CC,BC,CB':3,'-':4}

## Starter datasets

In [ ]:
# load and clean roma
# dataset roma contiene anche i dati aggiuntivi di IDA score
df_roma = pd.read_excel(file_roma_standardized)
df_roma['Sample ID'] = df_roma['patient_ID'].astype(str)+'_'+df_roma['embryo_ID'].astype(str)+'.'+df_roma['treatment_ID'].astype(str)
df_roma['class'] = df_roma['classification'].map({'eup':'eup','ane':'aneup','seg':'segm','ane_seg':'segm+aneup'})
df_roma = df_roma.replace('-',np.nan)
df_ida = pd.read_excel(file_roma_IDA)#,sheet_name='Cleavage features')
df_ida['ida score'] = df_ida['ida score'].replace('missing',None).astype(float)
df_ida.rename(columns={'operator\'s grading':'operators\' grading','ida score':'IDA Score'},inplace=True)
df_ida['grading_grouped'] = df_ida['operators\' grading'].map(grading_dict)
df_ida['grading_grouped_num'] = df_ida['grading_grouped'].map(grading_dict_num).fillna(4)
df_roma = pd.merge(df_roma,df_ida,on=['Sample ID'])

In [ ]:
df_roma.head(3)

In [ ]:
#load and clean unblind
df_unblind = pd.read_excel(file_roma_blind,sheet_name='unblinded')
df_unblind['IDA Score'] = df_unblind['IDA Score'].replace('-',None).astype(float)
df_unblind['grading_grouped'] = df_unblind['operators\' grading'].map(grading_dict)
df_unblind['grading_grouped_num'] = df_unblind['grading_grouped'].map(grading_dict_num).fillna(4)
df_unblind['class'] = df_unblind['eup, aneup, segm, segm+aneup'].map({0:'eup',1:'aneup',2:'segm',3:'segm+aneup'})
df_unblind = df_unblind.replace('-',np.nan)
df_unblind = df_unblind.rename(columns={'t9+':'t9'})
df_unblind_correction = pd.read_excel(file_roma_blind_correction)
for i,j in df_unblind_correction.iterrows():
    embryoID = j['ID_Emb']
    corrected_class = j['CheckClass eup ane seg aneseg']
    df_unblind.loc[df_unblind['ID_Emb']==embryoID,'class'] = corrected_class
df_unblind = df_unblind[df_unblind['class']!='?'].reset_index()
df_unblind['centre_ID'] = 'roma_unblind'
df_unblind['patient_ID'] = df_unblind['ID_Emb'].str.split('_').apply(lambda x: x[0])
df_unblind['treatment_ID'] = df_unblind['ID_Ciclo']
df_unblind['embryo_ID'] = df_unblind['ID_Emb']

#load and clean blind
df_blind = pd.read_excel(file_roma_blind,sheet_name='blinded')
df_blind['IDA Score'] = df_blind['IDA Score'].replace('-',None).astype(float)
df_blind['grading_grouped'] = df_blind['operators\' grading'].map(grading_dict)
df_blind['grading_grouped_num'] = df_blind['grading_grouped'].map(grading_dict_num).fillna(4)
df_blind = df_blind.replace('-',np.nan)
df_blind = df_blind.rename(columns={'t9+':'t9'})
df_blind['centre_ID'] = 'roma_blind'
df_blind['patient_ID'] = df_blind['ID_Emb'].str.split('_').apply(lambda x: x[0])
df_blind['treatment_ID'] = df_blind['ID_Ciclo']
df_blind['embryo_ID'] = df_blind['ID_Emb']

In [ ]:
df_unblind.head(3)

In [ ]:
df_blind.head(3)

In [ ]:
#load and clean valencia
df_valencia = pd.read_excel(file_valencia_standardized)
df_valencia['class'] = df_valencia['classification'].map({'eup':'eup','ane':'aneup','seg':'segm','ane_seg':'segm+aneup'})
df_valencia = df_valencia.replace('-',np.nan)

In [ ]:
df_valencia.head(3)

In [ ]:
#load and clean UK
df_uk = pd.read_excel(file_uk_standardized)
df_uk['class'] = df_uk['classification'].map({'eup':'eup','ane':'aneup','seg':'segm','ane_seg':'segm+aneup'})
df_uk = df_uk.replace('-',np.nan)

In [ ]:
df_uk.head(3)

In [ ]:
#load bologna
df_bologna = pd.read_excel(file_bologna)
df_bologna['centre_ID']='bologna'
df_bologna['treatment_ID'] = df_bologna['Sample ID'].str.split('_').apply(lambda x: x[0])
df_bologna['patient_ID'] = df_bologna['Sample ID'].str.split('_').apply(lambda x: x[0])
df_bologna['class'] = 'eup'
df_bologna.loc[df_bologna['Molecular Karyotype'].apply(lambda x: 'ANEUPLOIDE' in x),'class']='aneup'
df_bologna.loc[df_bologna['Molecular Karyotype'].apply(lambda x: 'aneup' in x),'class']='aneup'
df_bologna.loc[df_bologna['Molecular Karyotype'].apply(lambda x: 'SEGMENTAL' in x),'class']='segm'
df_bologna.loc[df_bologna['Molecular Karyotype'].apply(lambda x: ',' in x),'class']='segm+aneup'
df_bologna.loc[df_bologna['Molecular Karyotype'].apply(lambda x: ('+' in x)&('-' in x)),'class']='segm+aneup'

df_bologna[['Molecular Karyotype','class']].drop_duplicates()

In [ ]:
df_bologna.head(3)

In [ ]:
# contiene anche bologna fare attenzione
df_all = pd.concat([df_roma, df_valencia, df_uk, df_unblind,df_bologna])

display(df_all.shape) # dimensione dataset merged
display(df_all.centre_ID.value_counts()) # dimensioni centro specifico
display(df_all['class'].value_counts()) # conte delle classi su dataset merged
display(pd.crosstab(df_all.centre_ID, df_all['class']))

In [ ]:
df_all[['centre_ID','patient_ID','treatment_ID']].drop_duplicates().shape

In [ ]:
display(df_all['class'].value_counts(normalize=True))
display(pd.crosstab(df_all.centre_ID,df_all['class'],normalize='index'))

#### Missing timings

In [ ]:
df_all[[t + '_missing' for t in raw_feature_list]] = df_all[raw_feature_list].isna()
df_all['num_missing'] = df_all[[t + '_missing' for t in raw_feature_list]].sum(axis=1)
df_all = df_all.reset_index(drop=True)

In [ ]:
df_all.head(5)
#df_all['num_missing'].value_counts()

In [ ]:
sns.histplot(data=df_all,x='num_missing',hue='centre_ID',stat='count',common_norm=False,multiple='stack',hue_order=['IVIRMA','GeneraLife','Care-Fertility'],bins=range(15),shrink=1)
plt.axvline(5,c='r',ls='--')
plt.xlabel('Number of missing time features')
plt.grid()
plt.ylabel('Number of embryos')
plt.title('prova')
plt.show()

#plt.savefig(os.path.join(output_dir,'Missing_stat_num.jpeg'))

In [ ]:
# raw_feature_list = ['tPNa', 'tPNf', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 'tM', 'tSB', 'tB', 'tEB']
df_missing = df_all.groupby('centre_ID')[[t + '_missing' for t in raw_feature_list]].mean().reset_index()
df_missing

In [ ]:
df_missing_melt = df_missing.melt(id_vars='centre_ID')

sns.barplot(data=df_missing_melt,y='value',x='variable',hue='centre_ID',hue_order=['IVIRMA','GeneraLife','Care-Fertility'])
plt.xticks(range(len(raw_feature_list)),raw_feature_list,rotation=90)
#plt.grid()
plt.ylabel('Fraction of missing records')
plt.grid()
plt.title('prova')
plt.show()
#plt.savefig(os.path.join(output_dir,'Missing_stat.jpeg'))

#### Boxplots

In [ ]:
df_all_melt = df_all[['class']+raw_feature_list].melt(id_vars='class')

sns.boxplot(data=df_all_melt,y='value',x='variable',hue='class',hue_order=['eup','aneup','segm','segm+aneup'])
plt.xticks(range(len(raw_feature_list)),raw_feature_list,rotation=90)
plt.grid()
plt.ylabel('Elapsed time (h)')
plt.title('Multi-centers')
plt.title('prova')
plt.show()
#plt.savefig(os.path.join(output_dir,'boxplot_class.jpeg'))

In [ ]:
for myclass in df_all['class'].unique():
    df_all_melt = df_all.loc[df_all['class']==myclass,['centre_ID']+raw_feature_list].melt(id_vars='centre_ID')
    #print(myclass)
    plt.figure()
    sns.boxplot(data=df_all_melt,y='value',x='variable',hue='centre_ID',hue_order=['IVIRMA','GeneraLife','Care-Fertility'])
    plt.xticks(range(len(raw_feature_list)),raw_feature_list,rotation=90)
    plt.grid()
    plt.ylabel('Elapsed time (h)')
    plt.title(str(myclass))
    plt.show()
    #plt.savefig(os.path.join(output_dir,myclass+'.jpeg'))

#### Statistical tests

##### Kolmogorov Smirnov 2 samples

In [ ]:
raw_feature_list

In [ ]:
selected_feature_list

In [ ]:
dataset_dict

In [ ]:
df_ks2 = pd.DataFrame()
dataset_dict = {'GeneraLife':df_roma_processed,'Care-Fertility':df_uk_processed,'IVIRMA':df_valencia_processed,'allcenters':df_all_processed}

for center in dataset_dict.keys():
    dataset = dataset_dict[center]
    for t in tl_all.imputed_times:
    #for t in raw_feature_list + ['cc3_imp','cc2_imp','s2_imp','s3_imp','blast1_imp']:
        for c1 in ['eup','segm','aneup']:
            for c2 in ['eup','segm','aneup']:
                if c1 > c2:
                    list1 = dataset.loc[dataset['class']==c1, t].dropna()
                    list2 = dataset.loc[dataset['class']==c2, t].dropna()
                    effect = np.mean(list1) -np.mean(list2)

                    statistic, pvalue_ks = stats.ks_2samp(list1, list2)
                    statistic, pvalue_tind = stats.ttest_ind(list1,list2)
                    # statistic è un valore della statistica poco interpretabile 
                    # per tale motivo utilizziamo l'effetto medio dei tempi
                    df_ks2 = df_ks2.append({'center':center,
                                            'c1':c1,'c2':c2,
                                            'time':t,
                                            'effect':effect,
                                            'pvalue_ks':pvalue_ks,
                                            'pvalue_tind':pvalue_tind},
                                            ignore_index=True)

df_ks2['-log10p_ks'] = -np.log10(df_ks2['pvalue_ks'])
df_ks2['-log10p_tind'] = -np.log10(df_ks2['pvalue_tind'])
# non considerare il p_ttest che riguarda il test T indipendente a due campioni
# i nostri dati non sono parametrici

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
ax[0].set_title('KS 2 samples - Euploids vs Segmentals')
sns.barplot(ax=ax[0],data=df_ks2[(df_ks2.c1=='segm')&(df_ks2.c2=='eup')],x='time',y='-log10p_ks',hue='center',hue_order=['allcenters'])
ax[1].set_title('KS 2 samples - Euploids vs Aneuploids')
sns.barplot(ax=ax[1],data=df_ks2[(df_ks2.c1=='eup')&(df_ks2.c2=='aneup')],x='time',y='-log10p_ks',hue='center',hue_order=['allcenters'])
ax[2].set_title('KS 2 samples - Aneuploids vs Segmentals')
sns.barplot(ax=ax[2],data=df_ks2[(df_ks2.c1=='segm')&(df_ks2.c2=='aneup')],x='time',y='-log10p_ks',hue='center',hue_order=['allcenters'])


ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.show()
#plt.savefig(os.path.join(output_dir,'significancy_ks.jpeg'))

In [ ]:
df_ks2[(df_ks2.c1=='segm')&(df_ks2.c2=='eup')].head(30)

In [ ]:
df_ks2

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
ax[0].set_title('KS 2 samples - Segmental vs Euploid')
sns.barplot(ax=ax[0],data=df_ks2[(df_ks2.c1=='segm')&(df_ks2.c2=='eup')],x='time',y='-log10p_ks',hue='center')
ax[1].set_title('KS 2 samples - Euploid vs Aneuploid')
sns.barplot(ax=ax[1],data=df_ks2[(df_ks2.c1=='eup')&(df_ks2.c2=='aneup')],x='time',y='-log10p_ks',hue='center')
ax[2].set_title('KS 2 samples - Segmental vs Aneuploid')
sns.barplot(ax=ax[2],data=df_ks2[(df_ks2.c1=='segm')&(df_ks2.c2=='aneup')],x='time',y='-log10p_ks',hue='center')


ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()
plt.show()
#plt.savefig(os.path.join(output_dir,'significancy_ks_centers.jpeg'))

In [ ]:
display(df_ks2.c1.unique())
display(df_ks2.c2.unique())

#### Sibling analysis

In [ ]:
df_pairs = pd.merge(df_processed_all, df_processed_all, on=['centre_ID','treatment_ID','patient_ID'],suffixes=['_x','_y'])
df_pairs = df_pairs[df_pairs['class_x']>df_pairs['class_y']]

In [ ]:
df_pairs.head(3)

In [ ]:
for time in tl_all.imputed_times:
    df_pairs[time + '_delta'] = df_pairs[time + '_y'] - df_pairs[time + '_x']
    df_pairs[time + '_binary'] = df_pairs[time + '_y'] - df_pairs[time + '_x'] > 0
display(df_pairs.centre_ID.value_counts())

In [ ]:
df_pairs.groupby(['centre_ID','class_x','class_y'])[['patient_ID']].count()

In [ ]:
df_pairs.loc[(df_pairs['class_x']==c1)&(df_pairs['class_y']==c2)]

In [ ]:
df_sibling = pd.DataFrame()

dataset_dict = {'GeneraLife':df_roma_processed,'Care-Fertility':df_uk_processed,'IVIRMA':df_valencia_processed,'allcenters':df_all_processed}

for center in dataset_dict.keys():
    dataset = dataset_dict[center]
    df_pairs = pd.merge(dataset,dataset,on=['centre_ID','treatment_ID','patient_ID'],suffixes=['_x','_y'])
    df_pairs = df_pairs[df_pairs['class_x']!=df_pairs['class_y']]
    for time in tl_all.imputed_times:
        time_delta = time+'_delta'
        df_pairs[time_delta] = df_pairs[time+'_y'] - df_pairs[time+'_x']
        df_pairs[time+'_binary'] = df_pairs[time+'_y'] - df_pairs[time+'_x'] > 0
    
        for c1 in ['eup','segm','aneup']:
            for c2 in ['eup','segm','aneup']:
                if 1==1:#c1 > c2:
                    df_sel = df_pairs.loc[(df_pairs['class_x']==c1)&(df_pairs['class_y']==c2)]
                    if len(df_sel)>0:

                        list1 = df_sel[time_delta].dropna()
                        n_success = (list1>0).sum()
                        avg_delay = np.mean(list1)
                        n_trials = len(list1)
                        frac_binary = n_success/n_trials -0.5
                        list_zero = np.zeros(n_trials)

                        pvalue_bin = stats.binom_test(n_success, n=n_trials, p=0.5, alternative='two-sided')
                        statistic, pvalue_wilcoxon = stats.wilcoxon(list1,list_zero)
                        


                        df_sibling = df_sibling.append({'center':center,'c1':c1,'c2':c2,'time':time,'avg_delay':avg_delay,
                                                              'pvalue_bin':pvalue_bin,'pvalue_wilcoxon':pvalue_wilcoxon,'frac_binary':frac_binary},ignore_index=True)


df_sibling['-log10p_bin'] = -np.log10(df_sibling['pvalue_bin'])
df_sibling['-log10p_wil'] = -np.log10(df_sibling['pvalue_wilcoxon'])

In [ ]:
df_sibling

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
ax[0].set_title('Binomial test - Segmental vs Euploid')
sns.barplot(ax=ax[0],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='eup')],x='time',y='-log10p_bin',hue='center',hue_order=['allcenters'])
ax[1].set_title('Binomial test - Segmental vs Aneuploid')
sns.barplot(ax=ax[1],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='aneup')],x='time',y='-log10p_bin',hue='center',hue_order=['allcenters'])
ax[2].set_title('Binomial test - Euploid vs Aneuploid')
sns.barplot(ax=ax[2],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='aneup')],x='time',y='-log10p_bin',hue='center',hue_order=['allcenters'])

ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].tick_params(axis='x', labelrotation=45)
ax[1].tick_params(axis='x', labelrotation=45)
ax[2].tick_params(axis='x', labelrotation=45)
plt.tight_layout()
#plt.title('Binomial test - ')
plt.show()
# plt.savefig(os.path.join(output_dir,'significancy_sibling_bin.jpeg'))

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
plt.title('Significancy binomial test')
ax[0].set_title('Binomial test - Segmental vs Euploid')
sns.barplot(ax=ax[0],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='eup')],x='time',y='-log10p_bin',hue='center')
ax[1].set_title('Binomial test - Segmental vs Aneuploid')
sns.barplot(ax=ax[1],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='aneup')],x='time',y='-log10p_bin',hue='center')
ax[2].set_title('Binomial test - Euploid vs Aneuploid')
sns.barplot(ax=ax[2],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='aneup')],x='time',y='-log10p_bin',hue='center')
ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].tick_params(axis='x', labelrotation=45)
ax[1].tick_params(axis='x', labelrotation=45)
ax[2].tick_params(axis='x', labelrotation=45)
plt.tight_layout()
plt.show()
# plt.savefig(os.path.join(output_dir,'significancy_siblings_centers_bin.jpeg'))

##### Wilcoxon signed-rank test

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
ax[0].set_title('Wilcoxon paired - Euploids vs Segmentals')
sns.barplot(ax=ax[0],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='eup')],x='time',y='-log10p_wil',hue='center',hue_order=['allcenters'])
ax[1].set_title('Wilcoxon paired - Euploid vs Aneuploids')
sns.barplot(ax=ax[1],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='aneup')],x='time',y='-log10p_wil',hue='center',hue_order=['allcenters'])
ax[2].set_title('Wilcoxon paired - Aneuploids vs Segmentals')
sns.barplot(ax=ax[2],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='aneup')],x='time',y='-log10p_wil',hue='center',hue_order=['allcenters'])

ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[0].tick_params(axis='x', labelrotation=45)
ax[1].tick_params(axis='x', labelrotation=45)
ax[2].tick_params(axis='x', labelrotation=45)
plt.tight_layout()
#plt.ylim([0,15])
plt.show()
#plt.savefig(os.path.join(output_dir,'significancy_sibling_tind.jpeg'))

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10))
ax[0].set_title('Wilcoxon paired - Euploids vs Segmentals')
sns.barplot(ax=ax[0],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='eup')],x='time',y='-log10p_wil',hue='center')
ax[1].set_title('Wilcoxon paired - Euploids vs Aneuploids')
sns.barplot(ax=ax[1],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='aneup')],x='time',y='-log10p_wil',hue='center')
ax[2].set_title('Wilcoxon paired - Aneuploids vs Segmentals')
sns.barplot(ax=ax[2],data=df_sibling[(df_sibling.c1=='segm')&(df_sibling.c2=='aneup')],x='time',y='-log10p_wil',hue='center')


ax[0].axhline(2,c='k',ls='--')
ax[1].axhline(2,c='k',ls='--')
ax[2].axhline(2,c='k',ls='--')
ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[0].tick_params(axis='x', labelrotation=45)
ax[1].tick_params(axis='x', labelrotation=45)
ax[2].tick_params(axis='x', labelrotation=45)
plt.tight_layout()
plt.show()
#plt.savefig(os.path.join(output_dir,'significancy_siblings_centers_tind.jpeg'))

In [ ]:
df_sibling

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(16,10),sharey=True)
ax[0].set_title('Binomial test - Euploids vs Segmentals')
sns.barplot(ax=ax[0],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='segm')],x='time',y='frac_binary',hue='center')
ax[1].set_title('Binomial test - Euploids vs Aneuploids')
sns.barplot(ax=ax[1],data=df_sibling[(df_sibling.c1=='eup')&(df_sibling.c2=='aneup')],x='time',y='frac_binary',hue='center')
ax[2].set_title('Binomial test - Aneuploids vs Segmentals')
sns.barplot(ax=ax[2],data=df_sibling[(df_sibling.c1=='aneup')&(df_sibling.c2=='segm')],x='time',y='frac_binary',hue='center')

ax[0].axhline(.0,c='k',ls='-')
ax[1].axhline(.0,c='k',ls='-')
ax[2].axhline(.0,c='k',ls='-')

ax[0].axhline(.1,c='k',ls='--')
ax[1].axhline(.1,c='k',ls='--')
ax[2].axhline(.1,c='k',ls='--')

ax[0].axhline(-0.1,c='k',ls='--')
ax[1].axhline(-0.1,c='k',ls='--')
ax[2].axhline(-0.1,c='k',ls='--')
plt.tight_layout()
plt.ylim([-0.25,0.25])
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[0].set_yticks([-0.2,-0.1,0,0.1,0.2])
ax[0].set_yticklabels([0.3,0.4,0.5,0.6,0.7])
ax[1].set_yticks([-0.2,-0.1,0,0.1,0.2])
ax[1].set_yticklabels([0.3,0.4,0.5,0.6,0.7])
ax[2].set_yticks([-0.2,-0.1,0,0.1,0.2])
ax[2].set_yticklabels([0.3,0.4,0.5,0.6,0.7])
ax[0].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[1].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
ax[0].set_ylabel('segmental sibling\n is delayed wrt euploid sibling')
ax[1].set_ylabel('aneuploid sibling\n is delayed wrt euploid sibling')
ax[2].set_ylabel('segmental sibling\n is delayed wrt aneuploid sibling')
plt.show()
#plt.savefig(os.path.join(output_dir,'significancy_siblings_centers_frac.jpeg'))

#### Defining training and test sets
1. test 40%
2. training 60%

In [ ]:
df_roma_train0, df_roma_test0 = train_test_split(df_roma[df_roma['class'].isin(['segm','eup'])], test_size=0.4,random_state=104)
df_roma_train = df_roma_train0.copy()
df_roma_test = df_roma_test0.copy()

In [ ]:
display(df_roma_train.shape)
display(df_roma_test.shape)

In [ ]:
df_val_train0, df_val_test0 = train_test_split(df_valencia[df_valencia['class'].isin(['segm','eup'])], test_size=0.4,random_state=104)
df_val_train = df_val_train0.copy()
df_val_test = df_val_test0.copy()

In [ ]:
display(df_val_train.shape[0])
display(df_val_test.shape[0])

In [ ]:
df_uk_train0, df_uk_test0 = train_test_split(df_uk[df_uk['class'].isin(['segm','eup'])], test_size=0.4,random_state=104)
df_uk_train = df_uk_train0.copy()
df_uk_test = df_uk_test0.copy()

In [ ]:
display(df_uk_train.shape[0])
display(df_uk_test.shape[0])

In [ ]:
df_all_train = pd.concat([df_roma_train, df_val_train, df_uk_train])
df_all_test = pd.concat([df_roma_test, df_val_test, df_uk_test])

In [ ]:
#1 train 4 models: model specific and global model
tl_roma = TimeLapseAnalyzer()
tl_roma.train(df_roma_train.copy(), model_feature_list = selected_feature_list + ['ritardo_max','ritardo_min','pca_ll'], target='is_segmental')
tl_val = TimeLapseAnalyzer()
tl_val.train(df_val_train.copy(), model_feature_list = selected_feature_list + ['ritardo_max','ritardo_min','pca_ll'], target='is_segmental')
tl_uk = TimeLapseAnalyzer()
tl_uk.train(df_uk_train.copy(), model_feature_list = selected_feature_list + ['ritardo_max','ritardo_min','pca_ll'], target='is_segmental')
tl_all = TimeLapseAnalyzer(PCA_LL_TH=-80)
tl_all.train(df_all_train.copy(), model_feature_list = selected_feature_list + ['ritardo_max','ritardo_min','pca_ll'], target='is_segmental')

In [ ]:
df_roma_processed = tl_roma.process_data(df_roma, train=False)
df_uk_processed = tl_uk.process_data(df_uk, train=False)
df_valencia_processed = tl_roma.process_data(df_valencia, train=False)
df_all_processed = tl_all.process_data(df_all, train=False)

In [ ]:
df_roma_processed.head(3)

In [ ]:
df_uk_processed.head(3)

In [ ]:
df_valencia_processed.head(3)

In [ ]:
df_all_processed.head(3)

#### Models transferability

In [ ]:
model_dict = {'roma':tl_roma,'uk':tl_uk,'valencia':tl_val,'allcenters':tl_all}
dataset_dict = {'roma':df_roma_test,'uk':df_uk_test,'valencia':df_val_test,'allcenters':df_all_test}
# iterate on trained models and datasets
df_tralability = pd.DataFrame()
for modelname in model_dict.keys():
    model = model_dict[modelname]
    for dataname in dataset_dict.keys():
        data = dataset_dict[dataname]
        
        df_processed = model.process_data(data,train=False)
        predictions = model.predict(df_processed)
        df_processed['segmental_score'] = predictions


        df_sel = df_processed[(df_processed.FLAG_QC==False)&(df_processed['class'].isin(['segm','eup']))]
        y_test = df_sel['class']=='segm'
        y_pred_score = df_sel['segmental_score']
        logit_roc_auc = roc_auc_score(y_test, y_pred_score)
        df_tralability = df_tralability.append({'model':modelname,'data':dataname,'auroc':logit_roc_auc}, ignore_index=True)

# y_test riporta la classificazione in vero o falso
# y_pred_score sono le probabilità assegnate per fare la classificazione 
# entrmabi sono due vettori della lunghezza uguale
# logit_roc_auc, punteggio dell'area sotto la curva


In [ ]:
df_tralability

In [ ]:
df_tralability_pivot = df_tralability.pivot(index='model',columns='data')
df_tralability_pivot

In [ ]:
palette= sns.color_palette("vlag", as_cmap=True)
sns.heatmap(data = df_tralability_pivot, cmap = palette, annot = True)
plt.title('AUROC contingency table')
plt.show()
#plt.savefig(os.path.join(output_dir,'model_traslability.jpeg'))

#### ROC curves
##### ROC segmentals and euploids

In [ ]:
dataset_dict = {'GeneraLife':df_roma,'Care-Fertility':df_uk,'IVIRMA':df_valencia,'allcenters':df_all}


for center in dataset_dict.keys():
    dataset = dataset_dict[center]
    
    dataset_train0, dataset_test0 = train_test_split(dataset[dataset['class'].isin(['segm','eup'])], test_size=0.4,random_state=104)
    dataset_train = dataset_train0.copy()
    dataset_test = dataset_test0.copy()
    tl = TimeLapseAnalyzer()
    tl.train(dataset_train,
             model_feature_list = selected_feature_list+['pca_ll'],target='is_segmental')
    dataset_test_processed = tl.process_data(dataset_test,train=False)
    dataset_test_processed['segmental_score'] = tl.predict(dataset_test_processed)

    dataset_train_processed = tl.process_data(dataset_train,train=False)
    dataset_train_processed['segmental_score'] = tl.predict(dataset_train_processed)
    
    # eukutation on train set

    df_sel = dataset_train_processed[(dataset_train_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='segm'
    y_pred_score = df_sel['segmental_score']
    logit_roc_auc_train = roc_auc_score(y_test,y_pred_score)
    fpr_train, tpr_train, thresholds = roc_curve(y_test, y_pred_score)

    df_sel = dataset_test_processed[(dataset_test_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='segm'
    y_pred_score = df_sel['segmental_score']
    logit_roc_auc = roc_auc_score(y_test,y_pred_score)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_score)


    plt.figure(figsize=(6,5))
    plt.plot(fpr_train, tpr_train, label='Train Data (area = %0.3f)' % logit_roc_auc_train)
    plt.plot(fpr, tpr, label='Test Data (area = %0.3f)' % logit_roc_auc)

    #plt.plot(fpr_ida, tpr_ida, label='IDA Score (area = %0.3f)' % logit_roc_auc_ida)
    #plt.plot(fpr_grading, tpr_grading, label='Grading Score (area = %0.3f)' % logit_roc_auc_grading)

    plt.plot([-0.05, 1.05], [-0.05, 1.05],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])

    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(center)
    plt.legend(loc="lower right")
    plt.show()
    #plt.savefig(os.path.join(output_dir,'roc_segm_vs_eup_'+center+'.jpeg'))

##### ROC euploids vs aneuploid

In [ ]:
dataset_dict = {'GeneraLife':df_roma,'Care-Fertility':df_uk,'IVIRMA':df_valencia,'Multicenter':df_all}


for center in dataset_dict.keys():
    dataset = dataset_dict[center]
    
    dataset_train0, dataset_test0 = train_test_split(dataset[dataset['class'].isin(['aneup','eup'])], test_size=0.4,random_state=104)
    dataset_train = dataset_train0.copy()
    dataset_test = dataset_test0.copy()
    tl = TimeLapseAnalyzer()
    tl.train(dataset_train,
             model_feature_list = selected_feature_list+['pca_ll'],target='is_aneuploid')
    dataset_test_processed = tl.process_data(dataset_test,train=False)
    dataset_test_processed['aneup_score'] = tl.predict(dataset_test_processed)

    dataset_train_processed = tl.process_data(dataset_train,train=False)
    dataset_train_processed['aneup_score'] = tl.predict(dataset_train_processed)
    

    df_sel = dataset_train_processed[(dataset_train_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='aneup'
    y_pred_score = df_sel['aneup_score']
    logit_roc_auc_train = roc_auc_score(y_test,y_pred_score)
    fpr_train, tpr_train, thresholds = roc_curve(y_test, y_pred_score)

    df_sel = dataset_test_processed[(dataset_test_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='aneup'
    y_pred_score = df_sel['aneup_score']
    logit_roc_auc = roc_auc_score(y_test,y_pred_score)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_score)


    plt.figure(figsize=(6,5))
    plt.plot(fpr_train, tpr_train, label='Train Data (area = %0.3f)' % logit_roc_auc_train)
    plt.plot(fpr, tpr, label='Test Data (area = %0.3f)' % logit_roc_auc)
    plt.plot([-0.05, 1.05], [-0.05, 1.05],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])

    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(center)
    plt.legend(loc="lower right")
    plt.show()
    #plt.savefig(os.path.join(output_dir,'roc_aneup_vs_eup_'+center+'.jpeg'))

##### ROC segmentals and euploids+aneuploids

In [ ]:
dataset_dict = {'GeneraLife':df_roma,'Care-Fertility':df_uk,'IVIRMA':df_valencia,'Multicenter':df_all}


for center in dataset_dict.keys():
    dataset = dataset_dict[center]
    
    dataset_train0, dataset_test0 = train_test_split(dataset[dataset['class'].isin(['segm','aneup','eup'])], test_size=0.4,random_state=104)
    dataset_train = dataset_train0.copy()
    dataset_test = dataset_test0.copy()
    tl = TimeLapseAnalyzer()
    tl.train(dataset_train,
             model_feature_list = selected_feature_list+['pca_ll'],target='is_segmental')
    dataset_test_processed = tl.process_data(dataset_test,train=False)
    dataset_test_processed['segmental_score'] = tl.predict(dataset_test_processed)

    dataset_train_processed = tl.process_data(dataset_train,train=False)
    dataset_train_processed['segmental_score'] = tl.predict(dataset_train_processed)
    
    # eukutation on train set

    df_sel = dataset_train_processed[(dataset_train_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='segm'
    y_pred_score = df_sel['segmental_score']
    logit_roc_auc_train = roc_auc_score(y_test,y_pred_score)
    fpr_train, tpr_train, thresholds = roc_curve(y_test, y_pred_score)

    df_sel = dataset_test_processed[(dataset_test_processed.FLAG_QC==False)]
    y_test = df_sel['class']=='segm'
    y_pred_score = df_sel['segmental_score']
    logit_roc_auc = roc_auc_score(y_test,y_pred_score)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_score)
    
    plt.figure(figsize=(6,5))
    plt.plot(fpr_train, tpr_train, label='Train Data (area = %0.3f)' % logit_roc_auc_train)
    plt.plot(fpr, tpr, label='Test Data (area = %0.3f)' % logit_roc_auc)
    plt.plot([-0.05, 1.05], [-0.05, 1.05],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])

    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(center)
    plt.legend(loc="lower right")

    #plt.savefig(os.path.join(output_dir,'roc_segm_vs_aneup_or_eup_'+center+'.jpeg'))

#### ROC for centers

In [ ]:
#1 split datasets
df_all['is_GeneraLife'] = df_all['centre_ID'] == 'GeneraLife'
df_all['is_IVIRMA'] = df_all['centre_ID'] == 'IVIRMA'
df_all['is_Care-Fertility'] = df_all['centre_ID'] == 'Care-Fertility'


for center1,center2 in [('GeneraLife','IVIRMA'),('GeneraLife','Care-Fertility'),('IVIRMA','Care-Fertility')]:
    target = 'is_'+ center1

    df_center_train0, df_center_test0 = train_test_split(df_all[(df_all['class']=='segm')
                                                                &(df_all['centre_ID'].isin([center1,center2]))], 
                                                         test_size=0.4,random_state=104)
    df_center_train = df_center_train0.copy()
    df_center_test = df_center_test0.copy()


    #1 train 4 models: model specific and global model
    tl_center = TimeLapseAnalyzer()
    tl_center.train(df_center_train,model_feature_list = tl_center.imputed_times+['pca_ll'],target=target)
    df_center_train_processed = tl_center.process_data(df_center_train,train=False)
    df_center_test_processed = tl_center.process_data(df_center_test,train=False)
    df_center_train_processed['center_score'] = tl_center.predict(df_center_train_processed)
    df_center_test_processed['center_score'] = tl_center.predict(df_center_test_processed)
    
    # evalutation on test set
    plt.figure()
    y_pred_score = df_center_test_processed['center_score']
    y_test = df_center_test_processed[target]
    logit_roc_auc = roc_auc_score(y_test,y_pred_score)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_score)

    y_pred_score = df_center_train_processed['center_score']
    y_test = df_center_train_processed[target]
    logit_roc_auc_train = roc_auc_score(y_test,y_pred_score)
    fpr_train, tpr_train, thresholds = roc_curve(y_test, y_pred_score)

    plt.figure(figsize=(8,6))
    plt.title(center1+' vs '+center2)
    plt.plot(fpr, tpr, label='Test Data (area = %0.3f)' % logit_roc_auc)
    plt.plot(fpr_train, tpr_train, label='Train Data (area = %0.3f)' % logit_roc_auc_train)


    plt.plot([-0.05, 1.05], [-0.05, 1.05],'r--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])

    plt.xlabel('FPR')
    plt.ylabel('TPR')
    #plt.title('Curve ROC per centro')
    plt.legend(loc="lower right")
    #plt.savefig(os.path.join(output_dir,'roc_'+center1+'_vs_'+center2+'.jpeg'))

#### Filtering quality

In [ ]:
df_processed_all = tl_all.process_data(df_all, train=False)
predictions = tl_all.predict(df_processed_all)

In [ ]:
display(df_processed_all[['FLAG_QC','FLAG_QC_NUM_MISSING',
    'FLAG_QC_NOT_MONOTONY','FLAG_QC_MISSING_BLASTO','FLAG_QC_PCA']].mean())

display(df_processed_all.groupby('centre_ID')[['FLAG_QC','FLAG_QC_NUM_MISSING',
 'FLAG_QC_MISSING_BLASTO', 'FLAG_QC_NOT_MONOTONY','FLAG_QC_PCA']].mean())

display(df_processed_all.groupby('class')[['FLAG_QC','FLAG_QC_NUM_MISSING',
 'FLAG_QC_MISSING_BLASTO', 'FLAG_QC_NOT_MONOTONY','FLAG_QC_PCA']].mean())

In [ ]:
palette = sns.color_palette("rocket", as_cmap=True)
sns.scatterplot(data=df_processed_all, x='pca1', y='pca2', hue='pca_ll',s=50,palette=palette,alpha=0.7)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('papap')
plt.show()
#plt.savefig(os.path.join(output_dir,'pca_outlers_all_ll.jpeg'))


In [ ]:
sns.scatterplot(data=df_processed_all,x='pca1',y='pca2',hue='FLAG_QC_PCA',s=50,alpha=0.7)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('ajaja')
plt.show()
#plt.savefig(os.path.join(output_dir,'pca_outlers_all_QC.jpeg'))

In [ ]:
sns.scatterplot(data=df_processed_all[df_processed_all.centre_ID.isin(['IVIRMA','GeneraLife','Care-Fertility'])],x='pca1',y='pca2',hue='centre_ID',s=50,alpha=0.4,hue_order=['IVIRMA','GeneraLife','Care-Fertility'])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('sosos')
plt.show()
#plt.savefig(os.path.join(output_dir,'pca_outlers_all_centers.jpeg'))

In [ ]:
palette = sns.color_palette("rocket", as_cmap=True)
ax = sns.jointplot(x=df_processed_all['pca1'],y=df_processed_all['pca2'],
                   kind='scatter',
                   space=0,
                   alpha=0.5)

#### Outliers trajectories

In [ ]:
list_outliers = df_processed_all.sort_values('pca_ll').index[0:8].values

In [ ]:
list_outliers

In [ ]:
df_processed_all.columns

#### Delays trajectories
1. Full dataset
2. IVIRMA
3. Care-Fertility
4. GeneraLife

In [ ]:
#trajectories full dataset
dataset = df_all_processed[df_all_processed.FLAG_QC==False]
plotting_times = ['tPNa_imp', 'tPNf_imp', 't2_imp', 't3_imp', 't4_imp',
       't5_imp', 't6_imp', 't7_imp', 't8_imp', 't9_imp', 'tM_imp',
       'tSB_imp', 'tB_imp', 'tEB_imp']
plt.title('Delays trajectories in full and segmental aneuploidies - All centers')
plot_trajectory_new(dataset,plotting_times,outfile=os.path.join(output_dir,'trajectories_all.jpeg'))
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.ylim(0, 4)
plt.show()

In [ ]:
#trajectories val
dataset = df_valencia_processed[df_valencia_processed.FLAG_QC==False]
plotting_times = ['tPNa_imp', 'tPNf_imp', 't2_imp', 't3_imp', 't4_imp',
       't5_imp', 't6_imp', 't7_imp', 't8_imp', 't9_imp', 'tM_imp',
       'tSB_imp', 'tB_imp', 'tEB_imp']
plot_trajectory_new(dataset,plotting_times,outfile=os.path.join(output_dir,'trajectories_valencia.jpeg'))
plt.title('Delays trajectories in full and segmental aneuploidies - IVIRMA')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.ylim(0, 4)
plt.show()

In [ ]:
dataset = df_roma_processed[df_roma_processed.FLAG_QC==False]
plotting_times = ['tPNa_imp', 'tPNf_imp', 't2_imp', 't3_imp', 't4_imp',
       't5_imp', 't6_imp', 't7_imp', 't8_imp', 't9_imp', 'tM_imp',
       'tSB_imp', 'tB_imp', 'tEB_imp']
plot_trajectory_new(dataset,plotting_times,outfile=os.path.join(output_dir,'trajectories_roma.jpeg'))
plt.title('Delays trajectories in full and segmental aneuploidies - GeneraLife')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.ylim(0, 10)
plt.show()

In [ ]:
#trajectories uk
dataset = df_uk_processed[df_uk_processed.FLAG_QC==False]
plotting_times = ['tPNa_imp', 'tPNf_imp', 't2_imp', 't3_imp', 't4_imp',
       't5_imp', 't6_imp', 't7_imp', 't8_imp', 't9_imp', 'tM_imp',
       'tSB_imp', 'tB_imp', 'tEB_imp']
plot_trajectory_new(dataset,plotting_times,outfile=os.path.join(output_dir,'trajectories_uk.jpeg'))
plt.title('Delays trajectories in full and segmental aneuploidies - Care-Fertility')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.ylim(0, 4)
plt.show()